As empresas monitoradas pelo Monitriip, Sistema de Monitoramento de Transporte Coletivo de Passageiros da ANTT, são claramente empresas ativas, operantes de fato, uma vez que as viagens realizadas por seus ônibus são monitoradas por um equipamento embarcado que transmite tempestivamente (via 3G) vários dados (velocidade, tempo, localização, etc) à Agência de Transporte. Obter a lista dos CNPJs dessas empresas nos permitirá rotular algumas empresas da base nacional como existentes de fato.

Vamos obter os CNPJs das empresas monitoradas pela ANTT através de downloads de vários arquivos CSV disponíveis nos sites: https://dados.gov.br/dataset/monitriip-servico-regular-viagens e https://dados.gov.br/dataset/monitriip-servico-fretado-viagens 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import glob
import codecs

### Primeiro, vamos baixar os arquivos CSV dos dados de viagens do serviço de transporte regular:

In [2]:
# Obtém o html da página web que possui os links de todos os registros de viagens:
response = requests.get('https://dados.gov.br/dataset/monitriip-servico-regular-viagens')
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# Obtém os elementos <li> que possuem os links paras as páginas dos recursos:
lista_li = soup.find_all("span", {"class": "format-label", "data-format": "csv"})

In [4]:
# Obtém as URLs da página de cada recurso:
lista_url = list()
for li in lista_li:
    lista_url.append(li.parent.get('href'))

In [7]:
url_base = 'https://dados.gov.br'
lista_url_csv = list()

# Entre em cada página de recurso e obtém a url de download do respectivo csv:
for url_recurso in lista_url:
    url_completa = url_base + url_recurso
    print('Acessando: ', url_completa)
    
    # Abrir a página do recurso
    response = requests.get(url_completa)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar a url de download do csv:
    url_csv = soup.find(lambda tag:tag.name=="a" and '.CSV' in tag.get('href').upper()).get('href')
    print('Encontrada url: ', url_csv)
    lista_url_csv.append(url_csv)

print('Encontrados: ', len(lista_url_csv))
lista_url_csv

Acessando:  https://dados.gov.br/dataset/monitriip-servico-regular-viagens/resource/f3983ab1-5c74-4219-b498-eda7f62ce953
Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/f3983ab1-5c74-4219-b498-eda7f62ce953/download/viagem_regular_12_2020.csv
Acessando:  https://dados.gov.br/dataset/monitriip-servico-regular-viagens/resource/cef6b612-9a0e-4f38-a9ad-6424e6b5a5c2
Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/cef6b612-9a0e-4f38-a9ad-6424e6b5a5c2/download/viagem_regular_11_2020.csv
Acessando:  https://dados.gov.br/dataset/monitriip-servico-regular-viagens/resource/0661fa75-da66-4cee-8fe6-728afd6c8579
Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/0661fa75-da66-4cee-8fe6-728afd6c

Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/987d98cc-b7cf-4879-acfc-72c0de71b7fc/download/viagem_regular_01_2021.csv
Acessando:  https://dados.gov.br/dataset/monitriip-servico-regular-viagens/resource/ddef7c99-6916-4348-84ea-c8e8b424df56
Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/ddef7c99-6916-4348-84ea-c8e8b424df56/download/viagem_regular_02_2021.csv
Encontrados:  26


['http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/f3983ab1-5c74-4219-b498-eda7f62ce953/download/viagem_regular_12_2020.csv',
 'http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/cef6b612-9a0e-4f38-a9ad-6424e6b5a5c2/download/viagem_regular_11_2020.csv',
 'http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/0661fa75-da66-4cee-8fe6-728afd6c8579/download/viagem_regular_10_2020.csv',
 'http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/26ea6234-91fb-4978-a23d-09552ebc76d9/download/viagem_regular_09_2020.csv',
 'http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/c37b45ad-8

In [8]:
# Faz o download dos CSVs:
for url_csv in lista_url_csv:
    url_csv = url_csv.rsplit('url=', 1)[1]
    nome_arquivo = url_csv.rsplit('/', 1)[1]
    print('Baixando: ', nome_arquivo, ' de: ', url_csv)
    req = requests.get(url_csv)
    url_content = req.content
    csv_file = open('C:\\DATASETS\\Empresas Existentes de Fato\\ANTT\\' + nome_arquivo, 'wb')

    csv_file.write(url_content)
    csv_file.close()

Baixando:  viagem_regular_12_2020.csv  de:  https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/f3983ab1-5c74-4219-b498-eda7f62ce953/download/viagem_regular_12_2020.csv
Baixando:  viagem_regular_11_2020.csv  de:  https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/cef6b612-9a0e-4f38-a9ad-6424e6b5a5c2/download/viagem_regular_11_2020.csv
Baixando:  viagem_regular_10_2020.csv  de:  https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/0661fa75-da66-4cee-8fe6-728afd6c8579/download/viagem_regular_10_2020.csv
Baixando:  viagem_regular_09_2020.csv  de:  https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/26ea6234-91fb-4978-a23d-09552ebc76d9/download/viagem_regular_09_2020.csv
Baixando:  viagem_regular_08_2020.csv  de:  https://dados.antt.gov.br/dataset/a7bcf9d2-3659-4783-8a9d-d91c69204ebb/resource/c37b45ad-805a-4b9e-97f5-1181836f7088/download/viagem_regular_08_2020.csv
Baixando:  viag

Conseguimos baixar 26 arquivos CSV, correspondendo exatamente aos 26 recursos encontrados na página de dados abertos de registro de viagens regulares da ANTT.

### Agora, vamos baixar os arquivos CSV com os dados de viagens de serviço de transporte fretado:

In [9]:
# Obtém o html da página web que possui os links de todos os registros de viagens:
response = requests.get('https://dados.gov.br/dataset/monitriip-servico-fretado-viagens')
soup = BeautifulSoup(response.text, 'html.parser')

In [10]:
# Obtém os elementos <li> que possuem os links paras as páginas dos recursos:
lista_li = soup.find_all("span", {"class": "format-label", "data-format": "csv"})

In [11]:
# Obtém as URLs da página de cada recurso:
lista_url = list()
for li in lista_li:
    lista_url.append(li.parent.get('href'))

In [12]:
url_base = 'https://dados.gov.br'
lista_url_csv = list()

# Entre em cada página de recurso e obtém a url de download do respectivo csv:
for url_recurso in lista_url:
    url_completa = url_base + url_recurso
    print('Acessando: ', url_completa)
    
    # Abrir a página do recurso
    response = requests.get(url_completa)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar a url de download do csv:
    url_csv = soup.find(lambda tag:tag.name=="a" and '.CSV' in tag.get('href').upper()).get('href')
    print('Encontrada url: ', url_csv)
    lista_url_csv.append(url_csv)

print('Encontrados: ', len(lista_url_csv))
lista_url_csv

Acessando:  https://dados.gov.br/dataset/monitriip-servico-fretado-viagens/resource/18abd07e-5337-4005-bbbb-7156c355bec3
Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/18abd07e-5337-4005-bbbb-7156c355bec3/download/viagem_fretado_12_2020.csv
Acessando:  https://dados.gov.br/dataset/monitriip-servico-fretado-viagens/resource/30e5515d-9040-432c-ae4f-f8bdb7d488db
Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/30e5515d-9040-432c-ae4f-f8bdb7d488db/download/viagem_fretado_11_2020.csv
Acessando:  https://dados.gov.br/dataset/monitriip-servico-fretado-viagens/resource/ed76ea8a-1146-499d-a21e-8c395f39a222
Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/ed76ea8a-1146-499d-a21e-8c395f39

Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/0e5f44ec-4baa-4962-95a8-3d3963614588/download/viagem_fretado_01_2021.csv
Acessando:  https://dados.gov.br/dataset/monitriip-servico-fretado-viagens/resource/7c412ed5-8051-4f9d-8d39-a8ea6fdb020e
Encontrada url:  http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/7c412ed5-8051-4f9d-8d39-a8ea6fdb020e/download/viagem_fretado_02_2021.csv
Encontrados:  26


['http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/18abd07e-5337-4005-bbbb-7156c355bec3/download/viagem_fretado_12_2020.csv',
 'http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/30e5515d-9040-432c-ae4f-f8bdb7d488db/download/viagem_fretado_11_2020.csv',
 'http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/ed76ea8a-1146-499d-a21e-8c395f39a222/download/viagem_fretado_10_2020.csv',
 'http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/966d9f4b-4808-449e-bc64-a31923eb664d/download/viagem_fretado_09_2020.csv',
 'http://landpage-h.cgu.gov.br/dadosabertos/index.php?url=https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/ff137800-e

In [13]:
# Faz o download dos CSVs:
for url_csv in lista_url_csv:
    url_csv = url_csv.rsplit('url=', 1)[1]
    nome_arquivo = url_csv.rsplit('/', 1)[1]
    print('Baixando: ', nome_arquivo, ' de: ', url_csv)
    req = requests.get(url_csv)
    url_content = req.content
    csv_file = open('C:\\DATASETS\\Empresas Existentes de Fato\\ANTT\\' + nome_arquivo, 'wb')

    csv_file.write(url_content)
    csv_file.close()

Baixando:  viagem_fretado_12_2020.csv  de:  https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/18abd07e-5337-4005-bbbb-7156c355bec3/download/viagem_fretado_12_2020.csv
Baixando:  viagem_fretado_11_2020.csv  de:  https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/30e5515d-9040-432c-ae4f-f8bdb7d488db/download/viagem_fretado_11_2020.csv
Baixando:  viagem_fretado_10_2020.csv  de:  https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/ed76ea8a-1146-499d-a21e-8c395f39a222/download/viagem_fretado_10_2020.csv
Baixando:  viagem_fretado_09_2020.csv  de:  https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/966d9f4b-4808-449e-bc64-a31923eb664d/download/viagem_fretado_09_2020.csv
Baixando:  viagem_fretado_08_2020.csv  de:  https://dados.antt.gov.br/dataset/8ddd197d-81e6-43bb-a2b3-c9a158ddfbae/resource/ff137800-e866-465d-8688-3be1d9d06b68/download/viagem_fretado_08_2020.csv
Baixando:  viag

Conseguimos baixar 26 arquivos CSV, correspondendo exatamente aos 26 recursos encontrados na página de dados abertos de registro de viagens fretadas da ANTT.

### Agora, vamos obter os CNPJs das empresas tanto de serviço regular quanto de serviço fretado:

Conforme visto nos dicionários de dados das bases, existe a coluna 'cnpj' em todos os arquivos CSV. Vamos então obtê-la de cada arquivo e consolidar a lista de CNPJs de empresas que estão realizando serviços de transporte:

In [15]:
# IMPORTAÇÃO DOS ARQUIVOS CSV:

path = r'C:\DATASETS\Empresas Existentes de Fato\ANTT'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    print('Importando: ', filename)
    temp_df = pd.read_csv(filename, sep=';', dtype=str, usecols=['cnpj'])
    temp_df['Arquivo'] = filename
    li.append(temp_df)

# Concatena o resultado no dataframe
df = pd.concat(li, axis=0, ignore_index=True)

df.describe

Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_01_2019.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_01_2020.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_01_2021.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_02_2019.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_02_2020.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_02_2021.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_03_2019.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_03_2020.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_04_2019.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_04_2020.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_fretado_05_2019.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_

<bound method NDFrame.describe of                    cnpj                                            Arquivo
0        07222913000100  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...
1        23841160000169  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...
2        23338155000138  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...
3        03772611000146  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...
4        07661783000101  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...
...                 ...                                                ...
2088992  82647884000135  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...
2088993  82647884000135  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...
2088994  82647884000135  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...
2088995  76080738000178  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...
2088996  76080738000178  C:\DATASETS\Empresas Existentes de Fato\ANTT\v...

[2088997 rows x 2 columns]>

In [16]:
# Verificando tamanhos dos valores na coluna cnpj
df.cnpj.str.len().value_counts()

14    2088997
Name: cnpj, dtype: int64

Temos apenas CNPJ com 14 dígitos, conforme esperado. Não precisamos fazer limpeza ou correção.

In [17]:
# Verificando quantidade de CNPJ por arquivo:
display(df.Arquivo.value_counts())

# Verificando quantidade de CNPJ únicos:
print(df.cnpj.nunique())

C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_01_2020.csv    119227
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_12_2019.csv    114816
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_02_2020.csv    114130
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_10_2019.csv    104633
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_01_2021.csv    103352
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_11_2019.csv    101435
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_09_2019.csv    100626
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_08_2019.csv     97040
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_12_2020.csv     95119
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_05_2019.csv     94897
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_01_2019.csv     94720
C:\DATASETS\Empresas Existentes de Fato\ANTT\viagem_regular_03_2019.csv     92849
C:\DATASETS\Empr

1702


In [18]:
# Vamos ficar apenas com a coluna de cnpj e remover os duplicados:
df = df.drop(columns=['Arquivo'])
df.columns = ['CNPJ']
df.drop_duplicates(inplace=True)
print(df)

                   CNPJ
0        07222913000100
1        23841160000169
2        23338155000138
3        03772611000146
4        07661783000101
...                 ...
322442   17997420000197
463751   08324808000136
922772   05939969000146
1346903  04801028000189
1968457  02190197000102

[1702 rows x 1 columns]


Salvando o resultdo com a lista de CNPJs de empresas que fazem serviço de transporte coletivo de passageiros:

In [19]:
df.to_csv(r'C:\DATASETS\Empresas Existentes de Fato\empresas_transporte_coletivo.csv', index = False)